In [1]:
from os import path
from sys import argv
import pandas as pd
from sqlalchemy import create_engine

In [2]:

def to_title(col):
    return col.title().replace('_', '')
def to_tables(df):
    tables = []
    for col in df:
        if df[col].dtype not in ['int64', 'float64', 'bool', 'datetime64']:
            col_ids = df[col].factorize(sort = True)[0]
            tables.append(
                pd.DataFrame({
                    f'{col}_id': col_ids,
                    col: df[col]
                })
                .drop_duplicates()
                .sort_values(by = f'{col}_id')
                .reset_index(drop = True)
            )
            del df[col]
            df[f'{col}_id'] = col_ids
    tables.insert(0, df)
    return tables


In [3]:
filenames = [path.splitext(file_)[0] for file_ in argv[1:] if path.splitext(file_)[1] == '.csv']
for filename in filenames:
    raw = pd.read_csv(f"{filename}.csv")
    engine = create_engine(f'sqlite:///{filename}.db', echo = False)
    for i, df in enumerate(to_tables(raw)):
        df.to_sql(
            to_title('Observation' if i == 0 else df.columns[1]), # table name
            con = engine,
            index = False
        )